In [1]:
import numpy as np
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
def calculate_rsi(prices, dates, n=14):
    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed >= 0].sum() / n
    down = -seed[seed < 0].sum() / n
    rs = up / down
    rsi = {}

    for i in range(n, len(prices)):
        delta = deltas[i - 1]  # Cambio actual
        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta
        up = (up * (n - 1) + upval) / n
        down = (down * (n - 1) + downval) / n
        rs = up / down
        rsi[dates[i]] = 100. - 100. / (1. + rs)

    return rsi

# Define tu lista de acciones
acciones = ["TSLA", "F", "AAPL", "CCL", "AMZN", "AMD", "NVDA", "BAC", "T", "INTC",
            "META", "KMI", "SLB", "PFE", "XOM", "AAL", "GOOGL", "MSFT", "GOOG", "VZ",
            "MRO", "KEY", "CSX", "CSCO", "MU", "DAL", "KO", "CMCSA", "HBAN", "HAL",
            "WBD", "WFC", "C", "PARA", "HPE", "PYPL", "MRK", "OXY", "FCX", "WBA",
            "PCG", "NCLH", "VTRS", "GM", "BKR", "CTRA", "AMCR", "KDP", "DVN", "DIS",
            "SCHW", "PG", "USB", "JPM", "TFC", "BMY", "FIS", "MO", "RF", "MS", "SBUX",
            "HPQ", "NEE", "NKE", "COP", "CFG", "NWL", "JNJ", "ORCL", "ABBV", "NEM",
            "MDLZ", "CVS", "WMB", "CVX", "WMT", "GILD", "EQT", "BSX", "MOS", "PEP",
            "VICI", "QCOM", "FITB", "LUV", "V", "APA", "EXC", "HST", "KHC", "NI",
            "DOW", "GEN", "MNST", "BEN", "PSX", "UAL", "MDT", "ABT", "MCHP"]

# Crear un diccionario vacío para almacenar los datos de las acciones
data = {}

# Descargar los precios de cada acción y almacenarlos en el diccionario
for accion in acciones:
    data[accion] = yf.download(accion, start='2011-10-21', end='2023-07-09', interval="1d")

# Define tu capital inicial y costo fijo de inversión aquí
capital_inicial = 100000
costo_inversion = 5000

# Crear un diccionario para almacenar los resultados de cada acción
resultados_acciones = {}

for accion in acciones:
    # Obtener los precios de cierre y las fechas para la acción actual
    prices = data[accion]['Adj Close'].tolist()
    dates = data[accion].index.tolist()

    # Calcular el RSI para la acción actual
    rsi = calculate_rsi(prices, dates)

    # Obtener las señales de compra y venta basadas en el RSI (esto es un ejemplo, adapta según tu estrategia)
    signals = {}
    for date, rsi_value in rsi.items():
        if rsi_value < 30:
            signals[date] = 'Compra'
        elif rsi_value > 70:
            signals[date] = 'Venta'
        else:
            signals[date] = 'Neutral'

    # Inicializar variables y listas para el análisis de rendimiento para la acción actual
    capital_actual = capital_inicial
    posiciones = []
    rendimientos = []  # Lista para almacenar los rendimientos por operación para la acción actual
    num_operaciones_ganadoras = 0  # Contador de operaciones ganadoras
    num_operaciones_perdedoras = 0  # Contador de operaciones perdedoras

    # Recorrer todas las fechas para realizar las operaciones de compra y venta para la acción actual
    for i in range(40,len(dates)-1):
        date = dates[i]
        signal = signals.get(date, 'Neutral')

        if signal == 'Compra':
            # Calcula la cantidad de acciones que puedes comprar con el costo de inversión fijo
            cantidad_acciones = int(costo_inversion / prices[i])
            # Resta el costo de la inversión del capital disponible
            capital_actual -= costo_inversion
            # Agrega la posición a la lista de posiciones con el precio de compra
            posiciones.append({'Fecha': date, 'Cantidad': cantidad_acciones, 'Precio de compra': prices[i]})

        elif signal == 'Venta':
            for posicion in posiciones:
                cantidad_acciones = posicion['Cantidad']
                capital_actual += cantidad_acciones * prices[i]
                rendimiento = (prices[i] - posicion['Precio de compra']) / posicion['Precio de compra']
                rendimientos.append(rendimiento)

                if rendimiento >= 0:
                    num_operaciones_ganadoras += 1
                else:
                    num_operaciones_perdedoras += 1

            posiciones = []

    # Calcula el rendimiento acumulado para la acción actual
    rendimiento_acumulado = (capital_actual - capital_inicial) / capital_inicial

    # Calcula el rendimiento promedio de operaciones ganadoras para la acción actual
    if num_operaciones_ganadoras > 0:
        rendimiento_promedio_ganadoras = np.mean([rendimiento for rendimiento in rendimientos if rendimiento >= 0])
    else:
        rendimiento_promedio_ganadoras = 0

    # Calcula el rendimiento promedio de operaciones perdedoras para la acción actual
    if num_operaciones_perdedoras > 0:
        rendimiento_promedio_perdedoras = np.mean([rendimiento for rendimiento in rendimientos if rendimiento < 0])
    else:
        rendimiento_promedio_perdedoras = 0

    # Almacena los resultados de la acción actual en el diccionario de resultados
    resultados_acciones[accion] = {
        'Rendimiento acumulado': rendimiento_acumulado,
        'Número total de operaciones': num_operaciones_ganadoras + num_operaciones_perdedoras,
        'Número de operaciones ganadoras': num_operaciones_ganadoras,
        'Número de operaciones perdedoras': num_operaciones_perdedoras,
        'Rendimiento promedio de operaciones ganadoras': rendimiento_promedio_ganadoras,
        'Rendimiento promedio de operaciones perdedoras': rendimiento_promedio_perdedoras
    }

# Imprime los resultados para cada acción
for accion in acciones:
    print(f"Resultados para la acción {accion}:")
    print(f"Rendimiento acumulado: {resultados_acciones[accion]['Rendimiento acumulado']}")
    print(f"Número total de operaciones: {resultados_acciones[accion]['Número total de operaciones']}")
    print(f"Número de operaciones ganadoras: {resultados_acciones[accion]['Número de operaciones ganadoras']}")
    print(f"Número de operaciones perdedoras: {resultados_acciones[accion]['Número de operaciones perdedoras']}")
    print(f"Rendimiento promedio de operaciones ganadoras: {resultados_acciones[accion]['Rendimiento promedio de operaciones ganadoras']}")
    print(f"Rendimiento promedio de operaciones perdedoras: {resultados_acciones[accion]['Rendimiento promedio de operaciones perdedoras']}")
    print("-----------------------")

# Calcula y muestra el promedio de cada indicador para todas las acciones
promedio_rendimiento_acumulado = np.mean([resultados_acciones[accion]['Rendimiento acumulado'] for accion in acciones])
promedio_numero_total_operaciones = np.mean([resultados_acciones[accion]['Número total de operaciones'] for accion in acciones])
promedio_numero_operaciones_ganadoras = np.mean([resultados_acciones[accion]['Número de operaciones ganadoras'] for accion in acciones])
promedio_numero_operaciones_perdedoras = np.mean([resultados_acciones[accion]['Número de operaciones perdedoras'] for accion in acciones])
promedio_rendimiento_promedio_ganadoras = np.mean([resultados_acciones[accion]['Rendimiento promedio de operaciones ganadoras'] for accion in acciones])
promedio_rendimiento_promedio_perdedoras = np.mean([resultados_acciones[accion]['Rendimiento promedio de operaciones perdedoras'] for accion in acciones])

print("Promedio de indicadores para todas las acciones:")
print(f"Promedio de rendimiento acumulado: {promedio_rendimiento_acumulado}")
print(f"Promedio de número total de operaciones: {promedio_numero_total_operaciones}")
print(f"Promedio de número de operaciones ganadoras: {promedio_numero_operaciones_ganadoras}")
print(f"Promedio de número de operaciones perdedoras: {promedio_numero_operaciones_perdedoras}")
print(f"Promedio de rendimiento promedio de operaciones ganadoras: {promedio_rendimiento_promedio_ganadoras}")
print(f"Promedio de rendimiento promedio de operaciones perdedoras: {promedio_rendimiento_promedio_perdedoras}")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Resultados para la acción TSLA:
Rendimiento acumulado: 1.0783102881860733
Número total de operaciones: 79
Número de operaciones ganadoras: 71
Número de operaciones perdedoras: 8
Rendimiento promedio de operaciones ganadoras: 0.3288784057052257
Rendimiento promedio de operaciones perdedoras: -0.15010123147338744
-----------------------
Resultados para la acción F:
Rendimiento acumulado: 0.6406421497631073
Número total de operaciones: 142
Número de operaciones ganadoras: 107
Número de operaciones perdedoras: 35
Rendimiento promedio de operaciones ganadoras: 0.14474933735710296
Rendimiento promedio de operaciones perdedoras: -0.07274147695257223
-----------------------
Resultados para la acción AAPL:
Rendimiento acumulado: 0.26715403228759765
Número total de operaciones: 64
Número de operaciones ganadoras: 52
Número de operaciones perdedoras: 12
Rendimiento promedio de operaciones ganadoras: 0.13524729414258194
Rendimiento promedio de operaciones perdedoras: -0.12152420929833141
---------

In [3]:
# Define tu capital inicial y costo fijo de inversión aquí
capital_inicial = 100000
costo_inversion = 5000

# Calcular el RSI
rsi = calculate_rsi(prices, dates)
#rebalansear el valor del capitalfijo
# Obtener las señales de compra y venta basadas en el RSI (esto es un ejemplo, adapta según tu estrategia)
signals = {}
for date, rsi_value in rsi.items():
    if rsi_value < 30:
        signals[date] = 'Compra'
    elif rsi_value > 70:
        signals[date] = 'Venta'
    else:
        signals[date] = 'Neutral'

# Inicializar variables y listas para el análisis de rendimiento
capital_actual = capital_inicial
posiciones = []
operaciones = []

rendimientos = []  # Lista para almacenar los rendimientos por operación

# Recorrer todas las fechas para realizar las operaciones de compra y venta
for i in range(40,len(dates)-1):
    date = dates[i]
    signal = signals.get(date, 'Neutral')

    if signal == 'Compra':
        # Calcula la cantidad de acciones que puedes comprar con el costo de inversión fijo
        cantidad_acciones = int(costo_inversion / prices[i])
        # Resta el costo de la inversión del capital disponible
        capital_actual -= costo_inversion
        # Agrega la posición a la lista de posiciones con el precio de compra
        posiciones.append({'Fecha': date, 'Cantidad': cantidad_acciones, 'Precio de compra': prices[i]})

    elif signal == 'Venta':
        for posicion in posiciones:
            cantidad_acciones = posicion['Cantidad']
            precio_compra = posicion['Precio de compra']
            precio_venta = prices[i]
            rentabilidad = (precio_venta - precio_compra) / precio_compra
            rendimientos.append(rentabilidad)

            capital_actual += cantidad_acciones * precio_venta

        posiciones = []

# Calcula el rendimiento acumulado
rendimiento_acumulado = (capital_actual - capital_inicial) / capital_inicial

# Calcula la rentabilidad total considerando todas las operaciones realizadas
rentabilidad_total = np.prod(1 + np.array(rendimientos))

# Imprime la rentabilidad total después de vender todas las acciones compradas
print("Rentabilidad total después de vender todas las acciones compradas:", rentabilidad_total)

# Imprime las métricas adicionales según lo que necesites
print("Métricas:")
print("Rendimiento acumulado:", rendimiento_acumulado)

# Número total de operaciones realizadas (compras y ventas)
num_operaciones = len(rendimientos)
print("Número total de operaciones:", num_operaciones)

# Número de operaciones ganadoras y perdedoras
num_operaciones_ganadoras = len([rendimiento for rendimiento in rendimientos if rendimiento >= 0])
num_operaciones_perdedoras = len([rendimiento for rendimiento in rendimientos if rendimiento < 0])
print("Número de operaciones ganadoras:", num_operaciones_ganadoras)
print("Número de operaciones perdedoras:", num_operaciones_perdedoras)

# Rendimiento promedio de operaciones ganadoras y perdedoras
rendimiento_promedio_ganadoras = np.mean([rendimiento for rendimiento in rendimientos if rendimiento >= 0])
rendimiento_promedio_perdedoras = np.mean([rendimiento for rendimiento in rendimientos if rendimiento < 0])
print("Rendimiento promedio de operaciones ganadoras:", rendimiento_promedio_ganadoras)
print("Rendimiento promedio de operaciones perdedoras:", rendimiento_promedio_perdedoras)

Rentabilidad total después de vender todas las acciones compradas: 6461590.897348834
Métricas:
Rendimiento acumulado: 0.8317565275287628
Número total de operaciones: 85
Número de operaciones ganadoras: 82
Número de operaciones perdedoras: 3
Rendimiento promedio de operaciones ganadoras: 0.21855764514839285
Rendimiento promedio de operaciones perdedoras: -0.002446979405226969


In [4]:
# Crear el DataFrame con las fechas y las predicciones del RSI
df_rsi = pd.DataFrame({'Predicción RSI': rsi.values()})

# Montar Google Drive en Colab
from google.colab import drive
drive.mount('/content/drive')

# Guardar el DataFrame en un archivo Excel en Google Drive
output_path = '/content/drive/MyDrive/predicciones2.xlsx'
df_rsi.to_excel(output_path, header=False, index=False)

In [5]:
rsi

{Timestamp('2011-11-10 00:00:00'): 64.40398938103897,
 Timestamp('2011-11-11 00:00:00'): 67.68208251327592,
 Timestamp('2011-11-14 00:00:00'): 65.53717285862038,
 Timestamp('2011-11-15 00:00:00'): 67.03710455239877,
 Timestamp('2011-11-16 00:00:00'): 65.29808676592519,
 Timestamp('2011-11-17 00:00:00'): 55.944409055206286,
 Timestamp('2011-11-18 00:00:00'): 54.74771571109932,
 Timestamp('2011-11-21 00:00:00'): 51.52582434317472,
 Timestamp('2011-11-22 00:00:00'): 47.22051098865469,
 Timestamp('2011-11-23 00:00:00'): 44.12183250957455,
 Timestamp('2011-11-25 00:00:00'): 42.936542348112496,
 Timestamp('2011-11-28 00:00:00'): 47.605252477475865,
 Timestamp('2011-11-29 00:00:00'): 47.066610602926524,
 Timestamp('2011-11-30 00:00:00'): 57.186067304947066,
 Timestamp('2011-12-01 00:00:00'): 56.884295354292334,
 Timestamp('2011-12-02 00:00:00'): 55.55785668465684,
 Timestamp('2011-12-05 00:00:00'): 56.89787074727711,
 Timestamp('2011-12-06 00:00:00'): 56.82686553225955,
 Timestamp('2011-12-07